<h2>Finding Ghosts</h2>
<h2>Traccar 테스트 데이터 분석 - 데이터 수집량을 중심으로</h2>
<p>In theory, our tester's Traccar clients are supposed to send position data at every minutes. In fact, there is a delay of few seconds, practically. This is acceptable. However, we found that some clients didn't send data about a 3-6 hours.</p>
<p>We have to figure out why.</p>
<p>일부 환경에서 Traccar 가 오랜 기간 (몇 시간) 동안 데이터를 송신하지 않는 현상을 확인해야 한다.
주로 아이폰에서 일어나는 것으로 파악.</p>

<p>Date: 201901</p>
<p>Traccar Server: v4.2</p>
<p>Freq.: 60 sec.</p>
<table>
    <thead>
        <tr>
            <th>Name</th>
            <th>Phone</th>
            <th>OS</th>
            <th>Traccar Client</th>
        </tr>
    </thead>
    <tbody>
        <tr>
            <td>DBLAB0101</td>
            <td>V20</td>
            <td>8.0.0/F800K20i</td>
            <td>5.17</td>            
        </tr>
        <tr>
            <td>DBLAB0102</td>
            <td>S5</td>
            <td>6.0.1/F800K20i</td>
            <td>5.17</td>            
        </tr>
        <tr>
            <td>DBLAB0103</td>
            <td>iPhone 6</td>
            <td>12.1.2/16C104</td>
            <td>5.5</td>            
        </tr>
        <tr>
            <td>DBLAB0201</td>
            <td>iPhone XS</td>
            <td>12.1.2</td>
            <td>5.5</td>            
        </tr>
        <tr>
            <td>DBLAB0301</td>
            <td>iPhone X</td>
            <td>12.1.3 beta 4</td>
            <td>5.5</td>            
        </tr>
        <tr>
            <td>HCICG01</td>
            <td>?</td>
            <td>?</td>
            <td>?</td>            
        </tr>
    </tbody>
</table>

<p>어쩌다 알아내게 된 것: Traccar 클라이언트가 신호를 올바르게 보내지만 서버에 등록되어있지 않았던 경우 다시 등록하면 한방에 다 보낸다...</p>
<p>HCICG0160 실험자 자료 (1월 9일 종료)</p>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.io.json import json_normalize #package for flattening json in pandas df
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL
import json
import copy
from datetime import timedelta, date, timedelta

In [2]:
dbconfig = json.loads(open("res/dbconfig.properties").read())
engine = create_engine(URL(**dbconfig["cssc_jaunt"]))

In [3]:
sql = "select id, name, lastupdate from tc_devices;"
users = pd.read_sql_query(sql, engine)

In [4]:
users

,id,name,lastupdate
0,52,DBLAB0103,2019-01-25 09:05:08.970
1,51,DBLAB0102,2019-01-25 18:10:17.386
2,50,DBLAB0201,2019-01-25 17:37:59.779
3,54,HCICG01,2019-01-25 18:07:30.695
4,48,DBLAB0301,2019-01-18 09:47:04.167
5,49,DBLAB0101,2019-01-20 17:54:21.261


In [5]:
pos_query = "select d.name, p.* from tc_positions p, tc_devices d where p.deviceid = d.id and d.id IN (select id from tc_devices where name=ANY({$users}))"
users_str = users['name'].to_json(force_ascii=False, orient='values')
users_str

'["DBLAB0103","DBLAB0102","DBLAB0201","HCICG01","DBLAB0301","DBLAB0101"]'

In [6]:
#pgsql array style string : { ... }
users_str = list(users_str)
users_str[0] = '{'
users_str[-1] = '}'
users_str = "".join(users_str)
users_str

'{"DBLAB0103","DBLAB0102","DBLAB0201","HCICG01","DBLAB0301","DBLAB0101"}'

In [7]:
pos_query = "select d.name, p.* from tc_positions p, tc_devices d where p.deviceid = d.id and d.id IN (select id from tc_devices where name=ANY('{0}'))".format(users_str)
pos_query

'select d.name, p.* from tc_positions p, tc_devices d where p.deviceid = d.id and d.id IN (select id from tc_devices where name=ANY(\'{"DBLAB0103","DBLAB0102","DBLAB0201","HCICG01","DBLAB0301","DBLAB0101"}\'))'

In [8]:
#pos_query += "LIMIT 10"
df = pd.read_sql_query(pos_query, engine)

In [9]:
df.head()

,name,id,protocol,deviceid,servertime,devicetime,fixtime,valid,latitude,longitude,altitude,speed,course,address,attributes,accuracy,network
0,DBLAB0301,286386,osmand,48,2019-01-16 14:49:08.835,2019-01-16 14:49:06,2019-01-16 14:49:06,True,37.555166,127.050135,26.661201,0.0,0.0,None,"{""batteryLevel"":74.0,""distance"":0.0,""totalDist...",0.0,null
1,DBLAB0301,286387,osmand,48,2019-01-16 14:50:05.620,2019-01-16 14:49:40,2019-01-16 14:49:40,True,37.555168,127.050031,27.668663,0.0,0.0,None,"{""batteryLevel"":74.0,""distance"":9.18,""totalDis...",0.0,null
2,DBLAB0301,286388,osmand,48,2019-01-16 14:50:22.141,2019-01-16 14:50:05,2019-01-16 14:50:05,True,37.555143,127.050146,27.339186,0.0,0.0,None,"{""batteryLevel"":74.0,""distance"":10.52,""totalDi...",0.0,null
3,DBLAB0301,286389,osmand,48,2019-01-16 14:50:31.135,2019-01-16 14:50:28,2019-01-16 14:50:28,True,37.555248,127.050002,27.484837,0.0,0.0,None,"{""batteryLevel"":74.0,""distance"":17.27,""totalDi...",0.0,null
4,DBLAB0301,286390,osmand,48,2019-01-16 14:50:35.049,2019-01-16 14:50:35,2019-01-16 14:50:35,True,37.555233,127.049978,27.477942,0.0,0.0,None,"{""batteryLevel"":74.0,""distance"":2.7,""totalDist...",0.0,null


In [10]:
json.loads(df['attributes'][0])

{'batteryLevel': 74.0, 'distance': 0.0, 'totalDistance': 0.0, 'motion': False}

In [11]:
df["battery_level"] = df['attributes'].map(lambda x: json.loads(x)['batteryLevel'])
df["dist"] = df['attributes'].map(lambda x: json.loads(x)['distance'])
df["total_dist"] = df['attributes'].map(lambda x: json.loads(x)['totalDistance'])
df["motion"] = df['attributes'].map(lambda x: json.loads(x)['motion'])

In [12]:
df.head()

,name,id,protocol,deviceid,servertime,devicetime,fixtime,valid,latitude,longitude,...,speed,course,address,attributes,accuracy,network,battery_level,dist,total_dist,motion
0,DBLAB0301,286386,osmand,48,2019-01-16 14:49:08.835,2019-01-16 14:49:06,2019-01-16 14:49:06,True,37.555166,127.050135,...,0.0,0.0,None,"{""batteryLevel"":74.0,""distance"":0.0,""totalDist...",0.0,null,74.0,0.00,0.00,False
1,DBLAB0301,286387,osmand,48,2019-01-16 14:50:05.620,2019-01-16 14:49:40,2019-01-16 14:49:40,True,37.555168,127.050031,...,0.0,0.0,None,"{""batteryLevel"":74.0,""distance"":9.18,""totalDis...",0.0,null,74.0,9.18,9.18,False
2,DBLAB0301,286388,osmand,48,2019-01-16 14:50:22.141,2019-01-16 14:50:05,2019-01-16 14:50:05,True,37.555143,127.050146,...,0.0,0.0,None,"{""batteryLevel"":74.0,""distance"":10.52,""totalDi...",0.0,null,74.0,10.52,19.70,False
3,DBLAB0301,286389,osmand,48,2019-01-16 14:50:31.135,2019-01-16 14:50:28,2019-01-16 14:50:28,True,37.555248,127.050002,...,0.0,0.0,None,"{""batteryLevel"":74.0,""distance"":17.27,""totalDi...",0.0,null,74.0,17.27,36.97,False
4,DBLAB0301,286390,osmand,48,2019-01-16 14:50:35.049,2019-01-16 14:50:35,2019-01-16 14:50:35,True,37.555233,127.049978,...,0.0,0.0,None,"{""batteryLevel"":74.0,""distance"":2.7,""totalDist...",0.0,null,74.0,2.70,39.67,False


In [13]:
# attributes are flatted
# addresses are N/A in Korea
df.drop(["attributes", "address"], axis=1, inplace=True)

In [14]:
df.shape

(27122, 19)

In [15]:
cols = list(df)
nunique = df.apply(pd.Series.nunique)
cols_to_drop = nunique[nunique == 1].index
df.drop(cols_to_drop, axis=1, inplace=True)

In [16]:
df.head()

,name,id,deviceid,servertime,devicetime,fixtime,latitude,longitude,altitude,speed,course,accuracy,battery_level,dist,total_dist,motion
0,DBLAB0301,286386,48,2019-01-16 14:49:08.835,2019-01-16 14:49:06,2019-01-16 14:49:06,37.555166,127.050135,26.661201,0.0,0.0,0.0,74.0,0.00,0.00,False
1,DBLAB0301,286387,48,2019-01-16 14:50:05.620,2019-01-16 14:49:40,2019-01-16 14:49:40,37.555168,127.050031,27.668663,0.0,0.0,0.0,74.0,9.18,9.18,False
2,DBLAB0301,286388,48,2019-01-16 14:50:22.141,2019-01-16 14:50:05,2019-01-16 14:50:05,37.555143,127.050146,27.339186,0.0,0.0,0.0,74.0,10.52,19.70,False
3,DBLAB0301,286389,48,2019-01-16 14:50:31.135,2019-01-16 14:50:28,2019-01-16 14:50:28,37.555248,127.050002,27.484837,0.0,0.0,0.0,74.0,17.27,36.97,False
4,DBLAB0301,286390,48,2019-01-16 14:50:35.049,2019-01-16 14:50:35,2019-01-16 14:50:35,37.555233,127.049978,27.477942,0.0,0.0,0.0,74.0,2.70,39.67,False


In [17]:
#position by users

pos = {}
for idx, user in users.iterrows():
    pos[user['name']] = df[df['name'] == user['name']].sort_values(by="fixtime")

In [18]:
pos_original = copy.deepcopy(pos) #backup positions

In [31]:
# acutal data sent delay
'''in SQL
select d.name, p.id, p.devicetime, p.fixtime, p.attributes, p.devicetime - lag(p.devicetime) over (order by p.devicetime) as delay 
    from tc_positions p, tc_devices d 
    where p.deviceid = d.id and d.id IN (
        select id 
            from tc_devices 
            where name = ANY('{ __users__ }')
    )
'''
view_time = ['name', 'id', 'fixtime', 'devicetime', 'delay'] # view for device time and delay

for user in users['name']:
    pos[user]['delay'] = pos[user]['devicetime'].diff()    
    print(pos[user][view_time].head())

           name      id             fixtime          devicetime    delay
8192  DBLAB0103  294578 2019-01-20 23:34:49 2019-01-20 23:34:49      NaT
8193  DBLAB0103  294579 2019-01-20 23:35:29 2019-01-20 23:35:29 00:00:40
8194  DBLAB0103  294580 2019-01-20 23:36:30 2019-01-20 23:36:30 00:01:01
8195  DBLAB0103  294581 2019-01-20 23:38:36 2019-01-20 23:38:36 00:02:06
8196  DBLAB0103  294582 2019-01-20 23:39:38 2019-01-20 23:39:38 00:01:02
           name      id             fixtime          devicetime    delay
8120  DBLAB0102  294506 2019-01-20 21:52:58 2019-01-20 21:52:58      NaT
8121  DBLAB0102  294507 2019-01-20 21:53:59 2019-01-20 21:53:59 00:01:01
8122  DBLAB0102  294508 2019-01-20 21:55:02 2019-01-20 21:55:02 00:01:03
8123  DBLAB0102  294509 2019-01-20 21:56:02 2019-01-20 21:56:02 00:01:00
8124  DBLAB0102  294510 2019-01-20 21:57:02 2019-01-20 21:57:02 00:01:00
         name      id             fixtime          devicetime    delay
73  DBLAB0201  286459 2019-01-16 15:18:14 2019-01-16 

In [20]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [21]:
#experiment period by users

period = {}
for user in users['name']:
    period[user] = {}
    period[user]['start'], period[user]['end'] = pos[user]['devicetime'].min(), pos[user]['devicetime'].max()
    period[user]['delta'] = period[user]['end'] - period[user]['start']

period

{'DBLAB0103': {'start': Timestamp('2019-01-20 23:34:49'),
  'end': Timestamp('2019-01-25 09:05:07'),
  'delta': Timedelta('4 days 09:30:18')},
 'DBLAB0102': {'start': Timestamp('2019-01-20 21:52:58'),
  'end': Timestamp('2019-01-25 18:10:17'),
  'delta': Timedelta('4 days 20:17:19')},
 'DBLAB0201': {'start': Timestamp('2019-01-16 15:18:14'),
  'end': Timestamp('2019-01-25 17:37:57'),
  'delta': Timedelta('9 days 02:19:43')},
 'HCICG01': {'start': Timestamp('2019-01-17 18:26:46'),
  'end': Timestamp('2019-01-25 18:07:30'),
  'delta': Timedelta('7 days 23:40:44')},
 'DBLAB0301': {'start': Timestamp('2019-01-16 14:49:06'),
  'end': Timestamp('2019-01-18 09:47:03'),
  'delta': Timedelta('1 days 18:57:57')},
 'DBLAB0101': {'start': Timestamp('2019-01-16 14:55:42'),
  'end': Timestamp('2019-01-20 17:54:20'),
  'delta': Timedelta('4 days 02:58:38')}}

In [43]:
# get tracks
for user in users['name']:
    print(f"========================={user}=========================")
    df = pos[user].drop(pos[user].index[0])    
    df['delay'] = df['delay']//3600
    #df.groupby(df["delay"].dt.minute).count().plot(kind="bar")    
    #plt.show()    

=========================DBLAB0103=========================


AttributeError: 'Series' object has no attribute 'seconds'

In [23]:
# day iteration test
start_date, end_date = period['DBLAB0103']['start'], period['DBLAB0103']['end']
print(start_date, end_date)
days = []
for single_date in daterange(start_date, end_date):
    days.append(single_date.replace(hour=0, minute=0, second=0))
print(days)

2019-01-20 23:34:49 2019-01-25 09:05:07
[Timestamp('2019-01-20 00:00:00'), Timestamp('2019-01-21 00:00:00'), Timestamp('2019-01-22 00:00:00'), Timestamp('2019-01-23 00:00:00')]


In [ ]:
# delay mean by days
for user in users['name']:
    print(f"========================={user}=========================")
    for single_date in daterange(period[user]['start'], period[user]['end']):
        today = single_date.replace(hour=0, minute=0, second=0)
        tomorrow  = today + timedelta(days=1)
        print(f"DAY {today} - {tomorrow}")        
        
        df = pos[user]        
        pos_by_day = df[(df['devicetime'] >= today) & (df['devicetime'] < tomorrow)]
        print(pos_by_day[view_time].head(5))
        if pos_by_day.shape[0] == 0:
            continue
        
        print("Delays")        
        print("mean", pos_by_day['delay'].mean())   
        print("max", pos_by_day['delay'].max(), pos_by_day['delay'].idxmax(), pos_by_day.loc[[pos_by_day['delay'].idxmax()]])   
        print("min", pos_by_day['delay'].min())   
        print()
